In [40]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
def setdiff(x,y):
    return list(set(x)-set(y))

In [3]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')
train_df.columns = [col.replace('-','_').lower() for col in train_df.columns]
test_df.columns  = [col.replace('-','_').lower() for col in test_df.columns]

In [4]:
print('> 건수 : Train({:,}건), Test({:,}건)'.format(len(train_df),len(test_df)))
print('> Head of Data')
train_df.head()

> 건수 : Train(871,393건), Test(159,621건)
> Head of Data


,id,user_id,book_id,book_rating,age,location,book_title,book_author,year_of_publication,publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


In [5]:
#-----------------------------------------------------------------------------------------------#
# > 도서정보
#-----------------------------------------------------------------------------------------------#
# (1) ID : 샘플 고유 ID
#     - 모두 unique한 값들임
#     - train/test는 서로 다 다른 id들임
#     - 삭제해도 무관함
#
# (2) User-ID : 유저 고유 ID
#     - 한번씩만 평가한 유저도 있고, 11,143번이나 평가한 유저도 있음
#     - nunique : train(83,256건), test(21,909건)
#     - train에만있는대상(70,192건), test에만있는대상(8,845건)
#
# (3) Book-ID : 도서 고유 ID
#     - 한번씩만 평가된 책이 있고, 2,502번이나 평가된 책이 있음
#     - nunique : train(243,441건), test(62,333건)
#     - train에만있는대상(207,723건), test에만있는대상(26,615건)
#
# - (참조) https://dacon.io/competitions/official/236093/talkboard/408269?page=1&dtype=recent
# > Book-ID가 다르더라도 책의 세부 정보가 동일한 경우가 있습니다. 이러한 상황은 다음과 같은 경우에서 발생할 수 있습니다.
# 1. 다양한 출판사 및 발행국가 : 동일한 책이 여러 출판사에 의해 출간되거나, 다른 국가 혹은 지역에서 출간될 경우 서로 다른 Book-ID를 가질 수 있습니다.
# 2. 다양한 에디션 및 인쇄: 책의 개정판, 갱신판, 확장판 등이 발행될 때마다 새로운 Book-ID가 부여될 수 있습니다. 또한, 책의 여러 인쇄에서도 서로 다른 Book-ID가 사용될 수 있습니다.
# 3. 다양한 포맷: 동일한 제목의 책이 하드커버, 종이책, 오디오북, 전자책 등 다양한 형태로 출간될 경우, 각 포맷마다 고유한 Book-ID를 가지게 됩니다.
#
#-----------------------------------------------------------------------------------------------#
# > 유저정보
#-----------------------------------------------------------------------------------------------#
# (4) Age : 유저의 나이
#     - (최소,최대) = train(0,244), test(0,237)으로 데이터가 이상함
#     - train에서 0살이 495건(0.06%), 100살 초과가 2,573건(0.30%)으로, 총 3,068건(0.35%) 있음.
#     - 이것들을 어떻게 처리할지? (테스트에도 있어서 obs를 제거 할 수 없음.)
#     - user_id별로 age의 nunique는 1개씩임 (unique하지 않으면 채워넣으려고 했었는데..)
#
# (5) Location : 유저의 지역
#     - 유저지역이 1개인 곳도 있고, 12,267개인 곳도 있음
#     - nunique : train(20,971건), test(8,581건)
#     - train에만있는대상(13,897건), test에만있는대상(1,507건)
#
#-----------------------------------------------------------------------------------------------#
# > 도서정보
#-----------------------------------------------------------------------------------------------#
# (6) Book-Title : 도서명
#     - 도서명이 1개인 것도 있고, 2,502개인 것도 있음
#     - nunique : train(217,829건), test(59,408건)
#     - train에만있는대상(181,636건), test에만있는대상(23,215건)
#
# (7) Book-Author : 도서 저자
#     - 도서저자가 1개인 것도 있고, 8,467개인 것도 있음
#     - nunique : train(92,635건), test(32,605건)
#     - train에만있는대상(68,980건), test에만있는대상(8,950건) 
#
# (8) Year-Of-Publication : 도서 출판 년도 (-1일 경우 결측 혹은 알 수 없음)
#     - 동일한 도서명인데도 출판년도가 다른게 있음 (개정본 등의 이유로 보임)
#     - (최소,최대) = train(1376,2021), test(1909,2021)
#     - -1인경우가 train에서 11,515(1.32%), test에서 2,425(1.52%)
#
# (9) Publisher : 출판사
#     - 출판사가 1개인 것도 있고, 29,696개인 것도 있음
#     - nunique : train(15,505건), test(6,584건)
#     - train에만있는대상(10,123건), test에만있는대상(1,202건) 

#-----------------------------------------------------------------------------------------------#
# > 타겟정보
#-----------------------------------------------------------------------------------------------#
# (10) Book-Rating : 유저가 도서에 부여한 평점 (0점 ~ 10점)
#     - 0점은 inplict infomation(암시적 정보?)
#     - Rating이 0인 경우는 해당 유저가 특정 책에 관심이 없고, 관련이 없는 경우로 보고 0점도 예측 할 수 있도록 개발필요
#     - (참조) : https://dacon.io/competitions/official/236093/talkboard/408231?page=1&dtype=recent

In [6]:
# 아이디어
# (1) 책의 카테고리를 찾기

In [7]:
def string_to_numeric_infomation(data,string_columns,numeric_columns,agg=['mean','min','max']):
    if isinstance(string_columns,str):
        string_columns = [string_columns]
    if isinstance(numeric_columns,str):
        numeric_columns = [numeric_columns]
    
    new_data = data.copy()
    for str_col in tqdm(string_columns):
        cnt_data = new_data[str_col].value_counts().reset_index()\
            .rename(columns={'index':str_col,str_col:f'{str_col}_cnt'})
        rank_data = new_data[str_col].value_counts().rank(ascending=False).reset_index()\
            .rename(columns={'index':str_col,str_col:f'{str_col}_rank'})
        new_data = new_data\
            .merge(cnt_data,how='left',on=str_col)\
            .merge(rank_data,how='left',on=str_col)
        for num_col in numeric_columns:
            for _agg in agg:
                mean_data = new_data.groupby(str_col)[num_col].agg(_agg).reset_index()\
                    .rename(columns={num_col:f'{str_col}_{_agg}_{num_col}'})
                new_data = new_data\
                    .merge(mean_data,how='left',on=str_col)
        new_data.drop(str_col,axis=1,inplace=True)
        
    return new_data

In [8]:
train_df2 = train_df.copy()

# (1) feature들의 type 정리
unuse_features = ['id']
cat_features   = ['user_id','book_id','location','book_title','book_author','publisher']
num_features   = ['age','year_of_publication']
target_feature = 'book_rating'

# (2) 필요없는 컬럼 제거
train_df2.drop(columns=unuse_features,inplace=True)

# (3) categorical feature들을 numeric feature로 변환 (category가 너무 많아서 그대로 사용하기 힘듦)
train_df2 = string_to_numeric_infomation(data=train_df2,string_columns=cat_features,numeric_columns=num_features)

# (4) 모두 하나의 값을 가지는 컬럼 제거

100%|██████████| 6/6 [00:21<00:00,  3.53s/it]


In [9]:
print(train_df2.shape)
train_df2.head()

(871393, 51)


,book_rating,age,year_of_publication,user_id_cnt,user_id_rank,user_id_mean_age,user_id_min_age,user_id_max_age,user_id_mean_year_of_publication,user_id_min_year_of_publication,...,book_author_min_year_of_publication,book_author_max_year_of_publication,publisher_cnt,publisher_rank,publisher_mean_age,publisher_min_age,publisher_max_age,publisher_mean_year_of_publication,publisher_min_year_of_publication,publisher_max_year_of_publication
0,8,23.0,2001.0,8,11785.0,23.0,23.0,23.0,1990.125,1981.0,...,1976.0,2004.0,6510,22.0,37.939324,0.0,204.0,2000.402304,1992.0,2004.0
1,8,23.0,1981.0,8,11785.0,23.0,23.0,23.0,1990.125,1981.0,...,-1.0,2004.0,14299,10.0,35.642492,0.0,201.0,1992.328345,-1.0,2004.0
2,0,23.0,1981.0,8,11785.0,23.0,23.0,23.0,1990.125,1981.0,...,1972.0,2002.0,14299,10.0,35.642492,0.0,201.0,1992.328345,-1.0,2004.0
3,0,23.0,1991.0,8,11785.0,23.0,23.0,23.0,1990.125,1981.0,...,1990.0,2002.0,14797,9.0,37.492600,0.0,239.0,1995.910117,1964.0,2004.0
4,8,23.0,1989.0,8,11785.0,23.0,23.0,23.0,1990.125,1981.0,...,1988.0,1997.0,16018,8.0,36.905232,0.0,239.0,1996.427769,-1.0,2004.0


In [10]:
def delete_equal_columns(data):
    new_data = data.copy()
    
    try_iter = 0
    while True:
        try_iter+=1
        columns = new_data.columns
        equal_column_list = []
        
        pbar = trange(new_data.shape[1])
        for i in pbar:
            pbar.set_description('Try({}) '.format(try_iter))
            for j in range(new_data.shape[1]):
                if i>j:
                    col_i, col_j = columns[i], columns[j]

                    if new_data[col_i].nunique() == new_data[col_j].nunique():
                        ct = pd.crosstab(new_data[col_i],new_data[col_j])
                        if np.diag(ct).sum() == len(new_data):
                            equal_column_list.append([col_i,col_j])
                    else:
                        pass

        if len(equal_column_list)>0:
            delete_columns = pd.unique(np.array(equal_column_list)[:,-1])
            new_data = new_data.drop(columns=delete_columns)
        else:
            break
    
    return new_data

In [11]:
train_df3 = delete_equal_columns(train_df2)

Try(2) : 100%|██████████| 45/45 [00:13<00:00,  3.46it/s]


In [12]:
print(train_df2.shape,train_df3.shape)

(871393, 51) (871393, 45)


In [26]:
X = train_df3.drop(target_feature,axis=1)
y = train_df3[target_feature]

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=0,stratify=y)

In [41]:
model = CatBoostClassifier(
    #loss_function='RMSE',
    random_state=0,
    iterations=5000,
)
model.fit(
    X_train,y_train,
    eval_set=[(X_valid,y_valid)],
    metric_period=500,
)

Learning rate set to 0.066556
0:	learn: 2.1487805	test: 2.1486697	best: 2.1486697 (0)	total: 536ms	remaining: 44m 39s


KeyboardInterrupt: 

In [38]:
preds = model.predict(X_valid)
preds = [int(np.round(p,0)) for p in preds]
pd.crosstab(y_valid,preds)

col_0,-2,-1,0,1,2,3,4,5,6,7,8,9,10
book_rating,,,,,,,,,,,,,
0,3,246,15253,23892,24354,21716,16934,6224,906,185,29,18,1
1,0,0,5,20,39,52,89,42,10,2,0,2,0
2,0,0,6,33,48,100,142,64,8,2,1,0,0
3,0,0,16,45,108,210,323,149,21,3,0,0,0
4,0,0,6,68,163,301,495,221,29,8,1,0,0
5,0,0,57,344,1055,1936,2509,1465,224,45,12,34,2
6,0,0,33,248,667,1383,1923,896,141,33,10,0,0
7,0,1,79,529,1367,2697,3983,1916,397,159,32,10,1
8,0,0,95,697,1878,3459,5090,2951,705,266,125,127,1
